# Get current stock price data

In [20]:
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 

In [5]:
msft = yf.Ticker("MSFT")

In [9]:
#msft.history(period='6mo')

In [38]:
dataset_train = yf.download("MSFT", start="2022-01-27", end="2022-12-27") # poner un dia mas del actual

[*********************100%***********************]  1 of 1 completed


In [39]:
dataset_train.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-01-27 00:00:00-05:00,302.660004,307.299988,297.929993,299.839996,297.056519,53481300
2022-01-28 00:00:00-05:00,300.230011,308.480011,294.450012,308.260010,305.398346,49743700
2022-01-31 00:00:00-05:00,308.950012,312.380005,306.369995,310.980011,308.093140,46444500
2022-02-01 00:00:00-05:00,310.410004,310.630005,305.130005,308.760010,305.893738,40950400
2022-02-02 00:00:00-05:00,309.630005,315.119995,308.880005,313.459991,310.550079,36636000


In [40]:
training_set = dataset_train.iloc[:, 0:1].values
training_set.shape

(230, 1)

In [41]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))

In [42]:
training_set_scaled = sc.fit_transform(training_set)

In [43]:
X_train = []
y_train = []
for i in range(60, 127):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)
print(X_train.shape, y_train.shape)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
print(X_train.shape, y_train.shape)


(67, 60) (67,)
(67, 60, 1) (67,)


In [44]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense

In [45]:
model = Sequential()
model.add(LSTM(units=50,return_sequences=True,input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(optimizer='adam',loss='mean_squared_error')

In [46]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_8 (LSTM)               (None, 60, 50)            10400     
                                                                 
 dropout_8 (Dropout)         (None, 60, 50)            0         
                                                                 
 lstm_9 (LSTM)               (None, 60, 50)            20200     
                                                                 
 dropout_9 (Dropout)         (None, 60, 50)            0         
                                                                 
 lstm_10 (LSTM)              (None, 60, 50)            20200     
                                                                 
 dropout_10 (Dropout)        (None, 60, 50)            0         
                                                                 
 lstm_11 (LSTM)              (None, 50)               

# Training

In [47]:
model.fit(X_train,y_train,epochs=50,batch_size=32)

Epoch 1/50
3/3 [==============================] - 16s 345ms/step - loss: 0.1981
Epoch 2/50
3/3 [==============================] - 1s 159ms/step - loss: 0.0490
Epoch 3/50
3/3 [==============================] - 1s 177ms/step - loss: 0.0347
Epoch 4/50
3/3 [==============================] - 0s 179ms/step - loss: 0.0146
Epoch 5/50
3/3 [==============================] - 1s 179ms/step - loss: 0.0240
Epoch 6/50
3/3 [==============================] - 1s 182ms/step - loss: 0.0146
Epoch 7/50
3/3 [==============================] - 1s 168ms/step - loss: 0.0103
Epoch 8/50
3/3 [==============================] - 0s 114ms/step - loss: 0.0195
Epoch 9/50
3/3 [==============================] - 0s 110ms/step - loss: 0.0129
Epoch 10/50
3/3 [==============================] - 0s 147ms/step - loss: 0.0116
Epoch 11/50
3/3 [==============================] - 0s 154ms/step - loss: 0.0178
Epoch 12/50
3/3 [==============================] - 1s 197ms/step - loss: 0.0131
Epoch 13/50
3/3 [==============================]

## Predictions on Test set
I take the next month for making predictions

In [48]:
dataset_test = yf.download("MSFT", start="2022-12-28", end="2023-01-27") # poner un dia mas del actual

[*********************100%***********************]  1 of 1 completed


In [51]:
dataset_test.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-12-28 00:00:00-05:00,236.889999,239.720001,234.169998,234.529999,234.529999,17457100
2022-12-29 00:00:00-05:00,235.649994,241.919998,235.649994,241.009995,241.009995,19770700
2022-12-30 00:00:00-05:00,238.210007,239.960007,236.660004,239.820007,239.820007,21930800
2023-01-03 00:00:00-05:00,243.080002,245.750000,237.399994,239.580002,239.580002,25740000
2023-01-04 00:00:00-05:00,232.279999,232.869995,225.960007,229.100006,229.100006,50623400


In [52]:
real_stock_price = dataset_test.iloc[:, 0:1].values

In [54]:
dataset_test.shape, dataset_train.shape

((20, 6), (230, 6))

In [55]:
dataset_total = pd.concat((dataset_train['Open'], dataset_test['Open']), axis = 0)
dataset_total.shape

(250,)

In [ ]:
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs.shape